# Export metric results
- This script is used to export metric results of model performance based on hourly data.
- Simulations: GM_SLUCM, GM_CLMU. 


In [79]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import skew, kurtosis, 
home_path = "/gws/nopw/j04/duicv/yuansun/"

In [72]:
model_list = ['obs', 'wrf', 'wrf-ctsm']
station_list = ['hadisd', 'maqs', 'whitworth', 'sensor']
var_list = ['TAS', 'RH', 'WIND']
df_merged = pd.read_csv(f'{home_path}0_WRFvsWRF-CTSM/output_analysis/validation_four_sites/data_for_figure/merged_data.csv')
#df = df.dropna().copy()
df_merged['time'] = pd.to_datetime(df_merged['time'])
df_merged.head()

,time,TAS_obs,RH_obs,WIND_obs,station,TAS_wrf,RH_wrf,WIND_wrf,TAS_wrf-ctsm,RH_wrf-ctsm,WIND_wrf-ctsm
0,2022-01-01 00:00:00,13.0,81.153014,2.6,hadisd,12.483551,92.304764,6.215323,12.749115,90.818756,5.360440
1,2022-01-01 01:00:00,13.0,87.051121,3.6,hadisd,12.167511,92.512993,6.551469,12.536041,90.493980,5.537817
2,2022-01-01 02:00:00,13.0,87.051121,3.6,hadisd,12.134491,90.631203,7.758633,12.415405,89.124199,6.485768
3,2022-01-01 03:00:00,13.0,87.051121,2.6,hadisd,12.104614,88.701210,8.339851,12.375153,87.249580,7.023479
4,2022-01-01 04:00:00,13.0,81.153014,4.1,hadisd,12.014893,87.071274,8.254773,12.245209,85.874214,6.914871


In [94]:
def perkins_skill_score(observed, simulated, bins=100):
    # Create histogram counts for observed and simulated
    counts_obs, bin_edges = np.histogram(observed, bins=bins)
    counts_sim, _ = np.histogram(simulated, bins=bin_edges)
    
    # Compute minimum counts per bin
    min_counts = np.minimum(counts_obs, counts_sim)
    
    # Perkins Skill Score
    pss = min_counts.sum() / counts_obs.sum()
    return pss

def metrics(dataframe, var, station):
    if (station == 'sensor') and (var =='WIND'):
        return []
    else:
        df_station = dataframe[dataframe['station'] == station].copy()
        df_station_var = df_station[[f'{var}_obs', f'{var}_wrf', f'{var}_wrf-ctsm']].dropna().copy()
        df_station_var.reset_index(drop=True, inplace=True)

        results = []
        for model in ['wrf', 'wrf-ctsm']:
            observed = df_station_var[f'{var}_obs']
            simulated = df_station_var[f'{var}_{model}']
            # metrics
            mae = np.mean(np.abs(observed - simulated))
            mbe = np.mean(observed - simulated)
            rmse = np.sqrt(np.mean((observed - simulated) ** 2))
            corr = np.corrcoef(observed, simulated)[0, 1]
            nstd = np.std(simulated) / np.std(observed)
            skew_metrics = 1 - (skew(simulated) / skew(observed)) if skew(observed) != 0 else np.nan
            kurtosis_metrics = 1 - (kurtosis(simulated) / kurtosis(observed)) if kurtosis(observed) != 0 else np.nan
            pss_metrics = 1 - perkins_skill_score(observed, simulated)
            M5 = np.percentile(simulated, 5)
            O5 = np.percentile(observed, 5)
            abe_m5 = abs(M5 - O5)

            M95 = np.percentile(simulated, 95)
            O95 = np.percentile(observed, 95)
            abe_m95 = abs(M95 - O95)

            results.append({
            'Model': model,
            'Variable': var,
            'Station': station,
            'MAE': mae,
            'MBE': mbe,
            'RMSE': rmse,
            'CORR': corr,
            'NSTD': nstd,
            'SKEW': np.abs(skew_metrics),
            'KURTOSIS': np.abs(kurtosis_metrics),
            'PSS': pss_metrics,
            'ABE_m5': abe_m5,
            'ABE_m95': abe_m95
            })

        return results

In [95]:
all_results = []
for station in station_list:
    for var in var_list:
        all_results.extend(metrics(df_merged, var, station))

df_metrics = pd.DataFrame(all_results)
df_metrics.to_csv('data_for_figure/metrics.csv', index=False)
df_metrics

,Model,Variable,Station,MAE,MBE,RMSE,CORR,NSTD,SKEW,KURTOSIS,PSS,ABE_m5,ABE_m95
0,wrf,TAS,hadisd,1.130949,-0.229956,1.513669,0.969953,1.029710,0.171338,0.403602,0.534495,0.354195,0.902672
1,wrf-ctsm,TAS,hadisd,1.230251,-0.562309,1.576886,0.972624,1.055422,1.418611,0.609537,0.526243,0.514313,1.859245
2,wrf,RH,hadisd,8.867115,5.567587,11.823367,0.806687,1.031154,0.420391,20.486197,0.588930,3.210967,1.567181
3,wrf-ctsm,RH,hadisd,9.106611,6.306614,12.062595,0.815984,1.048847,0.164405,17.351917,0.625258,5.881169,5.451265
4,wrf,WIND,hadisd,1.867218,-1.601084,2.322733,0.727971,1.167446,0.673696,0.961227,0.677135,0.469510,1.801396
5,wrf-ctsm,WIND,hadisd,1.367104,-0.624516,1.801915,0.705445,1.113812,0.594597,1.096790,0.676905,0.250773,0.776997
6,wrf,TAS,maqs,1.139799,0.124485,1.518561,0.972936,1.074920,0.981287,0.262803,0.070095,0.775895,0.367057
7,wrf-ctsm,TAS,maqs,1.166179,-0.220343,1.495631,0.973822,1.070154,0.345724,0.743612,0.078605,0.210447,1.264933
8,wrf,RH,maqs,8.981012,7.054367,11.363272,0.863044,1.115176,0.564787,2.684566,0.209456,7.133176,0.608840
9,wrf-ctsm,RH,maqs,9.108132,7.012977,11.437389,0.857772,1.110008,0.311236,2.411864,0.166785,8.589677,3.667376


In [98]:
df_metrics_wrf = df_metrics[df_metrics['Model'] == 'wrf'].copy().reset_index(drop=True)
df_metrics_wrf_ctsm = df_metrics[df_metrics['Model'] == 'wrf-ctsm'].copy().reset_index(drop=True)
def check_better(model1_val, model2_val, metric):
    # MAE: mean absolute error (closer to 0 is better)
    # MBE: mean bias error (closer to 0 is better)
    # RMSE: root mean square error (closer to 0 is better)
    # CORR: correlation coefficient (closer to 1 is better)
    # NSTD: normalized standard deviation (closer to 1 is better)
    # PSS: potential skill score (closer to 0 is better)
    # ABE_m5: absolute bias error of 5th percentile (closer to 0 is better)
    # ABE_m95: absolute bias error of 95th percentile (closer to 0 is better)
    # SKEW: skewness (closer to 0 is better)
    # KURTOSIS: kurtosis (closer to 0 is better)
    # PSS: potential skill score (closer to 0 is better)
    if metric == 'MBE':
        if abs(model1_val) < abs(model2_val):
            return 'wrf'
        elif abs(model1_val) > abs(model2_val):
            return 'wrf-ctsm'
        else:
            return 'equal'
    elif metric == 'NSTD':
        if abs(model1_val - 1) < abs(model2_val - 1):
            return 'wrf'
        elif abs(model1_val - 1) > abs(model2_val - 1):
            return 'wrf-ctsm'
        else:
            return 'equal'   
    elif metric in ['MAE', 'RMSE', 'PSS', 'ABE_m5', 'ABE_m95', 'SKEW', 'KURTOSIS', 'PSS']:
        if model1_val < model2_val:
            return 'wrf'
        elif model1_val > model2_val:
            return 'wrf-ctsm'
        else:
            return 'equal'
    elif metric in ['CORR']:
        if model1_val > model2_val:
            return 'wrf'
        elif model1_val < model2_val:
            return 'wrf-ctsm'
        else:
            return 'equal'
    else:
        return 'unknown_metric'

metrics_to_check = ['MAE', 'MBE', 'RMSE', 'CORR', 'NSTD', 'SKEW', 'KURTOSIS', 'PSS', 'ABE_m5', 'ABE_m95']
better_metric_result = []
for station in station_list:
    for var in var_list:
        if (station == 'sensor') and (var == 'WIND'):
            continue
        df_wrf_row = df_metrics_wrf[(df_metrics_wrf['Station'] == station) & (df_metrics_wrf['Variable'] == var)]
        df_ctsm_row = df_metrics_wrf_ctsm[(df_metrics_wrf_ctsm['Station'] == station) & (df_metrics_wrf_ctsm['Variable'] == var)]
        for metric in metrics_to_check:
            better_model = check_better(df_wrf_row[metric].values[0], df_ctsm_row[metric].values[0], metric)
            better_metric_result.append({'variable': var, 'station': station, 'metric': metric, 'better_model': better_model})
df_better_metrics = pd.DataFrame(better_metric_result)
df_better_metrics.to_csv('data_for_figure/better_metrics.csv', index=False)
df_better_metrics

,variable,station,metric,better_model
0,TAS,hadisd,MAE,wrf
1,TAS,hadisd,MBE,wrf
2,TAS,hadisd,RMSE,wrf
3,TAS,hadisd,CORR,wrf-ctsm
4,TAS,hadisd,NSTD,wrf
...,...,...,...,...
105,RH,sensor,SKEW,wrf-ctsm
106,RH,sensor,KURTOSIS,wrf-ctsm
107,RH,sensor,PSS,wrf-ctsm
108,RH,sensor,ABE_m5,wrf
